## 6. Classification
### 1. Merge one hot encoded genres to lyrics

In [35]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
df_val = pd.read_csv('../dataset/Lyrics_en_artists_clean_test.csv', index_col=['Band'])
df

,Lyrics,'n,a,abstract,accordion,acid,acoustic,adoracao,adult,african,...,western,wonky,world,worship,wrestling,ye,zapstep,zolo,zouk,zydeco
Band,,,,,,,,,,,,,,,,,,,,,
Exhorder,youve given your life to decadent goals with a...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Heart,Hands on the wheel gridlock and steel dogs are...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Missing Persons,"I feel I was lost, never meant to be found, Fl...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elmore James,"I feel bad, I feel terrible, Im just as sad as...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lumidee,Could never accept reality in which I was pres...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Oasis,I live my life in the city Theres no easy way ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Carly Simon,The lies that you tell will leave you alone Th...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Trina,Da bitch is back I say da bitch is back Yeah d...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Donny Osmond,This is the moment; this is the day When I sen...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Label exploration  

In [2]:
%matplotlib inline

y_train = df.drop(['Lyrics'], axis=1)
y_test = df_val.drop(['Lyrics'], axis=1)
test_lyrics = df_val['Lyrics']
del df
del df_val
#pd.scatter_matrix(df, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

In [5]:
import seaborn as sns
sns.heatmap(y_train.corr())

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from sklearn.decomposition import PCA
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np

pca = PCA()
pca.fit(y_train)

# Dimension indexing
dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

# PCA components
components = pd.DataFrame(np.round(pca.components_, 4), columns = list(y_train.keys()))
components.index = dimensions

# PCA explained variance
ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
variance_ratios.index = dimensions

# Create a bar plot visualization
fig, ax = plt.subplots(figsize = (14,8))

# Plot the feature weights as a function of the components
components.plot(ax = ax, kind = 'bar');
ax.set_ylabel("Feature Weights")
ax.set_xticklabels(dimensions, rotation=0)


# Display the explained variance ratios
for i, ev in enumerate(pca.explained_variance_ratio_):
    ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

# Return a concatenated DataFrame
pd.concat([variance_ratios, components], axis = 1)

### 3. Doc2Vec Deep learning LSTM classifier

In [3]:
from gensim.models.doc2vec import Doc2Vec

doc2vec = Doc2Vec.load("../dataset/doc2vec")
doc_vectors = doc2vec.wv
print("Number of word vectors: {}".format(len(doc_vectors.vocab)))

C:\Users\juanp\Anaconda3\envs\mlnd\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Number of doc vectors: 61825


In [4]:
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['s', 'chorus'])

# python -m spacy download en
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    
def transform_lyric(lyric):
    lyric = simple_preprocess(str(lyric), deacc=True)

    new_lyric = list()
    for word in lyric:
        if(word not in stop_words):
            new_lyric.append(word)
    lyric = new_lyric


    lemma_lyric = list()
    doc = nlp(" ".join(lyric)) 
    lemma_lyric = list()
    for token in doc:
        if(token.pos_ in allowed_postags):
            lemma_lyric.append(token.lemma_)
    lyric = lemma_lyric
    return lyric

C:\Users\juanp\Anaconda3\envs\mlnd\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\juanp\Anaconda3\envs\mlnd\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
C:\Users\juanp\Anaconda3\envs\mlnd\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\juanp\Anaconda3\envs\mlnd\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\juanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import numpy as np
Text_INPUT_DIM = 100
train_size = y_train.shape[0] 
test_size = y_test.shape[0] 
text_train_arrays = np.zeros((train_size, Text_INPUT_DIM))
text_test_arrays = np.zeros((test_size, Text_INPUT_DIM))

for i in range(train_size):
    text_train_arrays[i] = doc2vec.docvecs[i]
    
for i in range(test_size):
    test_lyrics[i] = transform_lyric(test_lyrics[i])
    text_test_arrays[i] = doc2vec.infer_vector(test_lyrics[i])
#text_test_arrays = doc2vec.infer_vector(test_lyrics)
#del test_lyrics

In [6]:
from keras import backend as K
K.clear_session()

Using TensorFlow backend.


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

model_conv = Sequential()
model_conv.add(Embedding(input_dim=train_size, output_dim=100))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(y_train.shape[1], activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.summary()

estimator = model_conv.fit(text_train_arrays, y_train, validation_data=(text_test_arrays,y_test), epochs = 2, batch_size=1024)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         26693200  
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_2 (Dense)              (None, 646)               65246     
Total params: 26,856,510
Trainable params: 26,856,510
Non-trainable params: 0
________________________________________________________________

In [36]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
train_lyrics = df['Lyrics']
del df

In [73]:
#Bryce Fox
#Lucy
#genres: "indie poptimism","modern alternative rock","modern rock"

lyric = "So bring your vibe over here here here \
Throw it up in the air air air, oh \
I run my hands through your hair hair hair \
And give you that love cuz you're oh so rare \
You look better over here then take you do over there \
And I don't wanna share \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I'll fix you up something real real tall \
It's looking up, we should take it all off \
Yeah \
You remind me why the stars don't fall \
I swear I'll love you like your last name's Ball \
I'll take you to the lair if we make it past the stairs \
I don't wanna share, yeah \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
 \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
Yeah \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
(So bite your tongue before I bite it for ya) \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
She said she ain't Lucy \
I said not yet" 

lyric = transform_lyric(lyric)
print(lyric)

['bring', 'vibe', 'throw', 'air', 'air', 'air', 'run', 'hand', 'hair', 'hair', 'hair', 'give', 'love', 'rare', 'look', 'better', 'take', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'fix', 'something', 'real', 'real', 'tall', 'look', 'take', 'remind', 'star', 'fall', 'swear', 'love', 'last', 'name', 'ball', 'take', 'lair', 'make', 'past', 'stair', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'say', 'tryna', 'get', 'know', 'bite', 'tongue', 'bite', 'say', 'tryna', 'get', 'know

In [74]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)

sims = doc2vec.docvecs.most_similar([vector[0]], topn=2)
for sim in sims:
    print(train_lyrics[sim[0]], sim[1], sim[0])

Whats wrong nigga? I thought you was keeping it gangsta? I thought this what you wanted? They say if you scared go to church But remember He knows the bible too My baby when I get you get you get you get you Ima go head to ride with you Smoking lokin poking the deja till Im idle with you Cause I (want you) Now baby when Im riding here Im riding dirty Registration is out of service Smoking lokin drinking the potion you can see me swerving Cause I (want you) I want you more than you know I remember you took me to the mall last week baby You looked me in my eyes about 4 5 times Till I was hypnotized then you clarified That I (want you) You said Sherane aint got nothing on Lucy I said you crazy Roses are red violets are blue but me and you both pushing up daisies if I (want you) My baby when I get you get you get you get you Ima go head to ride with you Smoking lokin poking the deja till Im idle with you Cause I (want you) You said to me You said your name was Lucy I said wheres Ricardo? Y

In [75]:
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.4231725,
 'pop': 0.367642,
 'alternative': 0.13602054,
 'folk': 0.13572569,
 'mellow': 0.122727856,
 'jazz': 0.11584138,
 'dance': 0.11404512,
 'country': 0.11341774,
 'new': 0.11336716,
 'metal': 0.10036701}

In [68]:
lyric = "I hear you talkin' shit, bro you think you're the heat \
Please bow down to defeat you're barely mince meat \
Stop with the street talk, and start to do the street, walk \
Lock yourself in and tell me this, how you gonna battle with this sick shit that I spit bitch \
See you still flappin' your jaws, prepare to be thrown in the ocean \
In the middle with jaws \
Or prefer to be served and severed with claws, knock on the doors Of absolute death you may \
But please be ready for the automatic failures you may make \
So take this and wait for another dismembering, remembering \
The tethering of your dream and how it was minced to pieces \
Believe in yourself please, to save you now you'll be needin' \
Help from Jesus \
You think I'm copy, pastin' please, tell me how you're actin' up \
Better be quite or Imma bout to bitch smack your mouth shut \
For good, yeah man talk the hood talk better go back and walk the hood walk you ain't had the taste of a real G \
Get back down to your level and prepare to be beat \
Writin' on your sheet, like you're deep please bitch your weak \
You ain't on my level or a level at all, take a vacation \
Up the damn wall and fall \
Humpty Dumpty, I must be rusty but you better knock the shit Down if you goin' against me \
I'm hard to beat, you're hardly cheese compared to these beats And this emcee \
You ready, alright, so come on down, I bet you to do it \
Let's see if you got the balls or if you're just truant \
To me, yeah let's keep an eye, on your ass as the heat intensifies \
Talk to me in a smart tone, prepare to meet your demise \
Throw you, a hundred million miles is what im finna do \
Call me Son Goku \
I already told you, what I'm gonna do, finna plow through \
Yeah man try to tackle a topic more intense then mine \
I always rap from the heart, make the fake rappers sigh \
But I really really don't give at all a flying fuck \
My chance of making this shit is the same as seein' a flyin' truck \
Or it could all depend on some serious luck \
Maybe I'll release a track that is heard my millions \
Make it big and to the top and make billions \
But that's a dream for another day, I'm here to take you down \
Don't care what you say \
Freestylin' this rap shit to the grave \
Rap battlin' bitches in my spare time for loose change \
Please, all of you fake ass rappers are plan \
Make like like a plane and take the next trip away \
Far away, from here I don't need to here your shit anymore \
I'm tired of it now, next place to be, will be a fuckin' morgue \
But let me just let you all know this, I'm sick of it, all of it \
So listen next time I write another track \
Listen to the facts I spit, and find it through my old wack rap crap \
I know I've said some shit, that know I demise \
But just look at my own and, your own life \
But this time, through my eyes \
I got a couple little bitches that need a little call out \
But I'll reside, got more and more, don't get me wrong \
Now get the fuck back home and call your mom \
Continue to rap please, just leave me alone \
I'm done hearing wack disses like this \
I'm sick of this, listening to this petty as shit \
Bitch"
lyric = transform_lyric(lyric)
print(lyric)

['hear', 'talkin', 'shit', 'bro', 'think', 'heat', 'bow', 'defeat', 'barely', 'mince', 'meat', 'stop', 'street', 'talk', 'start', 'street', 'walk', 'lock', 'tell', 'go', 'battle', 'sick', 'shit', 'spit', 'bitch', 'see', 'still', 'flappin', 'jaw', 'prepare', 'throw', 'ocean', 'middle', 'jaw', 'prefer', 'serve', 'sever', 'claws', 'knock', 'door', 'absolute', 'death', 'may', 'please', 'ready', 'automatic', 'failure', 'may', 'make', 'take', 'wait', 'dismembering', 'remember', 'tethering', 'dream', 'mince', 'piece', 'believe', 'save', 'needin', 'help', 'think', 'copy', 'pastin', 'tell', 'actin', 'better', 'quite', 'imma', 'bout', 'bitch', 'smack', 'mouth', 'shut', 'good', 'talk', 'hood', 'talk', 'better', 'go', 'back', 'walk', 'hood', 'walk', 'taste', 'real', 'get', 'back', 'level', 'prepare', 'beat', 'writin', 'sheet', 'deep', 'bitch', 'weak', 'level', 'level', 'take', 'vacation', 'damn', 'wall', 'fall', 'humpty', 'dumpty', 'must', 'rusty', 'better', 'knock', 'shit', 'go', 'hard', 'beat', 

In [69]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)

sims = doc2vec.docvecs.most_similar([vector[0]], topn=2)
for sim in sims:
    print(train_lyrics[sim[0]], sim[1], sim[0])

[Royce Da 59] Once again relax, its just music. Niggaz right here, show you how I do Niggaz right here, show you my crew Yeah yeah yeah yeah yeah yeah yeah yeah yeah Yeah yeah Fuck a team like you who swing like you We block shots in the ring lightning Rock hot rocks kill, get the cream still Blasphemous mind ill steal rhyme skill Nas couldnt rhyme for this, Nickel-Nine will I go through, cool out, blow a whole crew Cool out bro, out-glow a whole jewel We bout to blow up, got your nose up You could catch a blocka-blocka try to stop or hold us And your block a whole bust, live news Ride through with one girl and five dudes Best crew in the D, niggaz best move All you niggaz gun sleep and your vest used Niggaz bluffin, bore me, nothin for me The only overlord me, only glory, you reach! Wake up and smell the aroma nigga you sleep The contract is out on The King, nigga you breach D-Elite - Jah, Cut Throat Billy Nix, Little, Nickle, Cha, upmost Respect dawgs, expect your neck cut rope The b

In [71]:
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

{'rock': 0.4231725,
 'pop': 0.367642,
 'alternative': 0.13602054,
 'folk': 0.13572569,
 'mellow': 0.122727856,
 'jazz': 0.11584138,
 'dance': 0.11404512,
 'country': 0.11341774,
 'new': 0.11336716,
 'metal': 0.10036701}